In [ ]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

import pickle

In [ ]:
BASE_PATH = ''
DATASET_NAME = ''

In [ ]:
def split_data(df, split=0.3):
  y = df['label']
  X = df.drop(columns=['label'])
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)

  return X_train, X_test, y_train, y_test

In [ ]:
def create_grid():
    # Number of trees in random forest
    n_estimators = [3200, 5000]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [100, 500, 1000]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [100, 500, 1000]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [2, 4, 10, 25, 50]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]# Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    return random_grid

In [ ]:
def load_data(path):
  df = pd.read_csv(path, sep=';', encoding='utf-8')
  return df

In [ ]:
def train(path, name, binary=False):
    if binary:
        df = create_balanced_dataset(load_data(path))
    else:
        df = load_data(path)
    
    print(df['label'].value_counts())

    X_train, X_test, y_train, y_test = split_data(df)

    random_grid = create_grid()
    
    rf = RandomForestClassifier()
    
    rf_random = RandomizedSearchCV(
        estimator=rf,
        param_distributions=random_grid,
        n_iter=25,
        cv=3,
        verbose=2,
        random_state=161194,
        n_jobs=-1
    )
    
    rf_random.fit(X_train, y_train)
    
    print('Score: ', rf_random.best_score_)
    print('Estimator: ', rf_random.best_estimator_)
    
    y_pred = rf_random.best_estimator_.predict(X_test)
    
    print("Accuracy: ", accuracy_score(y_test, y_pred))
    
    
    cm = pd.DataFrame(confusion_matrix(y_test, y_pred))
    cm_norm = pd.DataFrame(confusion_matrix(y_test, y_pred, normalize='true'))
    
    print(cm)
    print(cm_norm)
    
    cm.to_csv(BASE_PATH + r'results/' + name + 'cm.csv', sep=';', encoding='utf-8', index=False)
    cm_norm.to_csv(BASE_PATH + r'results/' + name + 'norm_cm.csv', sep=';', encoding='utf-8', index=False)
    
    with open(BASE_PATH + r'results/' + f'{name}model.pkl', 'wb') as file:
        pickle.dump(rf_random, file)

In [ ]:
def create_balanced_dataset(df):
    df_0 = df[df['label'] == 0]
    df_1 = df[df['label'] == 1].sample(n=2500)
    df_2 = df[df['label'] == 2].sample(n=2500)
    
    return pd.concat([df_0, df_1, df_2]).replace(2, 1)

In [ ]:
%%time
train(BASE_PATH + f'128_QF95-{DATASET_NAME}.csv', f'RF_128_QF-95_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'128_QF98-{DATASET_NAME}.csv', f'RF_128_QF-98_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'128_QF100-{DATASET_NAME}.csv', f'RF_128_QF-100_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'64_QF95-{DATASET_NAME}.csv', f'RF_64_QF-95_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'64_QF98-{DATASET_NAME}.csv', f'RF_64_QF-98_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'64_QF100-{DATASET_NAME}.csv', f'RF_64_QF-100_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'64_asc-{DATASET_NAME}.csv', f'RF_64_ASC_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'64_desc-{DATASET_NAME}.csv', f'RF_64_DESC_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'128_asc-{DATASET_NAME}.csv', f'RF_128_ASC_{DATASET_NAME}_')

In [ ]:
%%time
train(BASE_PATH + f'128_desc-{DATASET_NAME}.csv', f'RF_128_DESC_{DATASET_NAME}_')